In [93]:
import boto3
import os
import datetime
import pytz
import re
from tqdm import tqdm
import time
import wandb
from torch.utils.tensorboard import SummaryWriter
import shutil
import json
import base64
import fiftyone as fo


In [94]:
start_date = datetime.datetime(2023, 11, 19)    # year, month, day   
end_date = datetime.datetime(2023, 11, 25)      # 25 November 2023: Michigan vs Ohio
cameras_excluded = []

# Path to download the data
storage_target_root = "/media/dbogdoll/Datasets/data_engine_rolling"
data_target = os.path.join(storage_target_root, "data")
os.makedirs(data_target, exist_ok=True)

log_target = os.path.join(storage_target_root, "logs")
os.makedirs(log_target, exist_ok=True)

In [95]:
# Connect to AWS
with open("/home/dbogdoll/mcity_data_engine/.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        os.environ[key] = value

s3 = boto3.client(
    "s3",
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID", None),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY", None),
)

In [96]:
cameras = {
    "Geddes_Huron_1",
    "Geddes_Huron_2",
    "Huron_Plymouth_1",
    "Huron_Plymouth_2",
    "Main_stadium_1",
    "Main_stadium_2",
    "Plymouth_Beal",
    "Plymouth_Bishop",
    "Plymouth_EPA",
    "Plymouth_Georgetown",
    "State_Ellsworth_NE",
    "State_Ellsworth_NW",
    "State_Ellsworth_SE",
    "State_Ellsworth_SW",
    "Fuller_Fuller_CT",
    "Fuller_Glazier_1",
    "Fuller_Glazier_2",
    "Fuller_Glen",
    "Dexter_Maple_1",
    "Dexter_Maple_2",
    "Hubbard_Huron_1",
    "Hubbard_Huron_2",
    "Maple_Miller_1",
    "Maple_Miller_2",    
}

cameras_dict = {camera.lower(): {} for camera in cameras}

for id, camera in enumerate(cameras_dict):
    if camera not in cameras_excluded:
        cameras_dict[camera]["id"] = id
        cameras_dict[camera]["aws-sources"] = {}

print(f"Processed {len(cameras_dict)} cameras")

Processed 24 cameras


In [97]:
def process_aws_result(result):
    if 'CommonPrefixes' in result:
        folders = [prefix['Prefix'] for prefix in result['CommonPrefixes']] # Get list of folders from AWS response
        return folders
    else:
        return None

In [98]:
aws_sources = {
    "sip-sensor-data":[""],
    "sip-sensor-data2":["wheeler1/","wheeler2/"],
}

for bucket in tqdm(aws_sources, desc="Processing AWS sources"):
    for folder in aws_sources[bucket]:
        # Get and pre-process AWS data
        result = s3.list_objects_v2(Bucket=bucket, Prefix=folder, Delimiter='/')
        folders = process_aws_result(result)

        #Align folder names with camera names
        folders_aligned = [re.sub(r'(?<!_)(\d)', r'_\1', folder.lower().rstrip('/')) for folder in folders] # Align varying AWS folder names with camera names
        folders_aligned = [folder.replace('fullerct', 'fuller_ct') for folder in folders_aligned]  # Replace "fullerct" with "fuller_ct" to align with camera names
        folders_aligned = [folder.replace('fullser', 'fuller') for folder in folders_aligned]  # Fix "fullser" typo in AWS

        # Check cameras for AWS sources
        if folders:
            for camera_name in cameras_dict:
                for folder_name, folder_name_aligned in zip(folders, folders_aligned):
                    if camera_name in folder_name_aligned and "gs_" in folder_name_aligned: # gs_ is the prefix used in AWS
                        aws_source = f"{bucket}/{folder_name}"
                        if aws_source not in cameras_dict[camera_name]["aws-sources"]:
                            cameras_dict[camera_name]["aws-sources"][aws_source] = {}
        else:
            print(f"AWS did not return a list of folders for {bucket}/{folder}")
            print(result)



Processing AWS sources: 100%|██████████| 2/2 [00:00<00:00,  6.54it/s]


In [99]:
def select_data_test():
    "Select a single test file to download for each AWS source"
    n_cameras = 0
    n_aws_sources = 0
    n_files_to_download = 0
    download_size_bytes = 0
    for camera in cameras_dict:
        n_cameras += 1
        for aws_source in cameras_dict[camera]["aws-sources"]:
            n_aws_sources += 1
            file_downloaded_test = False
            bucket = aws_source.split("/")[0]
            prefix_camera = "/".join(aws_source.split("/")[1:])
            result = s3.list_objects_v2(Bucket=bucket, Prefix=prefix_camera, Delimiter='/')
            folders_day = process_aws_result(result)        # Each folder represents a day
            for folder_day in folders_day:
                date = folder_day.split('/')[-2]
                cameras_dict[camera]["aws-sources"][aws_source][date] = {}
                result = s3.list_objects_v2(Bucket=bucket, Prefix=folder_day, Delimiter='/')
                folders_hour = process_aws_result(result)    # Each folder represents an hour
                for folder_hour in folders_hour:
                    result = s3.list_objects_v2(Bucket=bucket, Prefix=folder_hour, Delimiter='/')
                    files = result["Contents"]
                    for file in files:
                        n_files_to_download += 1
                        download_size_bytes += file["Size"]
                        file_name = os.path.basename(file["Key"])
                        cameras_dict[camera]["aws-sources"][aws_source][date][file_name] = {}
                        cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["key"] = file["Key"]
                        cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["size"] = file["Size"]
                        cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["date"] = file["LastModified"].strftime("%Y-%m-%d %H:%M:%S")
                        file_downloaded_test = True
                        print(f"{aws_source} : {file_name}")
                        break # for file in files:
                    if file_downloaded_test:
                        break   # for folder_hour in folders_hour:
                if file_downloaded_test:
                    break   # for folder_day in folders_day:

    log_file_path = os.path.join("cameras_dict.json")
    with open(log_file_path, 'w') as json_file:
        json.dump(cameras_dict, json_file, indent=4)  

    print(f"Found {n_cameras} cameras")
    print(f"Found {n_aws_sources} AWS sources")
    print(f"Found {n_files_to_download} files to download")
    return n_files_to_download, download_size_bytes

In [100]:
def select_data():
    n_files_to_download = 0
    download_size_bytes = 0
    for camera in tqdm(cameras_dict, desc="Looking for data entries in range"):
        for aws_source in cameras_dict[camera]["aws-sources"]:
            bucket = aws_source.split("/")[0]
            prefix_camera = "/".join(aws_source.split("/")[1:])
            result = s3.list_objects_v2(Bucket=bucket, Prefix=prefix_camera, Delimiter='/')
            folders_day = process_aws_result(result)        # Each folder represents a day
            for folder_day in folders_day:
                date = folder_day.split('/')[-2]
                timestamp = datetime.datetime.strptime(date, '%Y-%m-%d')
                if start_date <= timestamp <= end_date:     # Only collect data within the date range
                    cameras_dict[camera]["aws-sources"][aws_source][date] = {}
                    result = s3.list_objects_v2(Bucket=bucket, Prefix=folder_day, Delimiter='/')
                    folders_hour = process_aws_result(result)    # Each folder represents an hour
                    for folder_hour in folders_hour:
                        result = s3.list_objects_v2(Bucket=bucket, Prefix=folder_hour, Delimiter='/')
                        files = result["Contents"]
                        for file in files:
                            n_files_to_download += 1
                            download_size_bytes += file["Size"]
                            file_name = os.path.basename(file["Key"])
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name] = {}
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["key"] = file["Key"]
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["size"] = file["Size"]
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["date"] = file["LastModified"].strftime("%Y-%m-%d %H:%M:%S")

    print(f"Found {n_files_to_download} files to download")
    return n_files_to_download, download_size_bytes

In [101]:
n_files_to_download, download_size_bytes = select_data_test()

sip-sensor-data/gs_Plymouth_Bishop/ : sip-delivery-stream-1-2023-03-13-18-56-11-ad8348dc-4dd4-3f8c-9403-add27be0c8b8
sip-sensor-data2/wheeler1/gs_Plymouth_Bishop/ : sip-data-stream2-delivery-stream-2-2024-04-04-18-39-25-01d61059-af25-3c57-bbc6-d2bb2fb2e9bf
sip-sensor-data2/wheeler2/gs_Maple_Miller_1/ : sip-data-stream2-delivery-stream-2-2024-04-12-17-23-12-b55b9f37-364a-39c2-803d-ab58bb86f711
sip-sensor-data2/wheeler1/gs_Fullser_FullerCT/ : sip-data-stream2-delivery-stream-2-2024-04-12-16-08-52-b39d2153-114e-3f15-8e67-66651d357768
sip-sensor-data/gs_State_Ellsworth_SE/ : sip-delivery-stream-1-2023-08-30-20-10-15-c22c084d-a7f2-3cbc-96b4-3d2b5401fff3
sip-sensor-data2/wheeler1/gs_State_Ellsworth_SE/ : sip-data-stream2-delivery-stream-2-2024-03-07-18-14-02-80bb8ed5-f4c5-39fb-b68f-4ba535a64aa9
sip-sensor-data2/wheeler2/gs_Fuller_Glen/ : sip-data-stream2-delivery-stream-2-2024-04-12-17-23-24-790869bc-fb2c-3526-9db1-da080c40697d
sip-sensor-data/gs_Main_stadium2/ : sip-delivery-stream-1-2023-0

In [102]:
def safety_checks():
    # Safety checks prior to downloading
    passed_checks = True

    # Check if each camera was assigned at least one AWS source
    for camera in cameras_dict:
        if len(cameras_dict[camera]["aws-sources"]) == 0:
            print(f"Camera {camera} has no AWS sources")
            passed_checks = False

    # Check if the total size of the data to download is within the limit
    MAX_SIZE_TB = 1.5
    total_size_tb = download_size_bytes / (1024 ** 4)
    if total_size_tb > MAX_SIZE_TB:
        print(f"Total size {total_size_tb} TB exceeds {MAX_SIZE_TB} TB limit")
        passed_checks = False
    else:
        print(f"Total size {total_size_tb} TB is within {MAX_SIZE_TB} TB limit")

    # Check if the date range is within the limit
    MAX_DAYS = 7
    days_delta = (end_date - start_date).days + 1 # +1 to include the end date
    if (days_delta) > MAX_DAYS:
        print(f"Date range of {days_delta} days exceeds {MAX_DAYS} days limit")
        passed_checks = False
    else:
        print(f"Date range of {days_delta} days is within {MAX_DAYS} days limit")
    return passed_checks

passed_checks = safety_checks()
print(passed_checks)

Total size 0.0030873126233927906 TB is within 1.5 TB limit
Date range of 7 days is within 7 days limit
True


In [103]:
# Download data
def download_data(delete_old_data=False):
    mb_per_s_list = []
    
    if passed_checks:
        download_successful = True

        if delete_old_data:
            try:
                shutil.rmtree(data_target)
            except:
                pass
            os.makedirs(data_target) 

        step = 0
        writer = SummaryWriter(log_dir="logs/download/s3")
        download_started = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with tqdm(desc="Downloading data", total=n_files_to_download) as pbar:
            for camera in cameras_dict:
                for aws_source in cameras_dict[camera]["aws-sources"]:
                    bucket = aws_source.split("/", 1)[0]
                    for date in cameras_dict[camera]["aws-sources"][aws_source]:
                        for file in cameras_dict[camera]["aws-sources"][aws_source][date]:
                            time_start = time.time()
                            
                            # AWS S3 Download
                            file_name = os.path.basename(file)
                            key = cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["key"]
                            target = os.path.join(data_target, file_name)
                            s3.download_file(bucket, key, target)
                            
                            # Calculate duration and GB/s
                            file_size_mb = cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["size"] / (1024 ** 2)
                            time_end = time.time()
                            duration = time_end - time_start
                            mb_per_s = file_size_mb / duration
                            mb_per_s_list.append(mb_per_s)
                            
                            # Update stats
                            step += 1
                            pbar.update(1)
                            writer.add_scalar("download/mb_per_second", mb_per_s, step)
        writer.close()

        # Check if all files were downloaded properly
        downloaded_files = os.listdir(data_target)

        for camera in tqdm(cameras_dict, desc="Checking downloaded data"):
            for aws_source in cameras_dict[camera]["aws-sources"]:
                for date in cameras_dict[camera]["aws-sources"][aws_source]:
                    for file in cameras_dict[camera]["aws-sources"][aws_source][date]:
                        file_name = os.path.basename(file)
                        if file not in downloaded_files:
                            download_successful = False
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["download_successful"] = False
                            print(f"File {file} was not downloaded properly")
                        else:
                            cameras_dict[camera]["aws-sources"][aws_source][date][file_name]["download_successful"] = True
        
        if download_successful: # Save log only if data is fully downloaded
            log = {}
            log["selection_start_date"] = start_date.strftime("%Y-%m-%d")
            log["selection_end_date"] = end_date.strftime("%Y-%m-%d")
            log["old_files_deleted"] = delete_old_data
            log["n_files_to_download"] = n_files_to_download
            log["download_size_tb"] = download_size_bytes / (1024 ** 4)
            log["download_started"] = download_started
            log["download_ended"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log["download_speed_avg_mbs"] = sum(mb_per_s_list) / len(mb_per_s_list)
            log["data"] = cameras_dict
            log_name = download_started.replace(" ", "_").replace(":", "_") + ".json"
            log_file_path = os.path.join(log_target, log_name)
            with open(log_file_path, 'w') as json_file:
                json.dump(log, json_file, indent=4)  
        else:
            print("Download failed")
    else:
        download_successful = False
        print("Safety checks failed. Not downloading data")

    return download_successful

run = wandb.init(
    name=f"{start_date.strftime('%Y%m%d')}_to_{end_date.strftime('%Y%m%d')}",
    sync_tensorboard=True,
    group="S3",
    job_type="download",
    project="Data Engine Download",
)

download_successful = download_data(delete_old_data=True)
print(download_successful)
run.finish(exit_code=0)


Checking downloaded data: 100%|██████████| 24/24 [00:00<00:00, 162622.45it/s]


True


download/mb_per_second,▆▆▆▆▆▁▅▅▁▆▆▆▇▇▅█▃▁▄▂▂▅▅▁▇▃▇▄▅▇▆▂▆▁▇▇▄▂
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
download/mb_per_second,9.96038
global_step,38


In [104]:
def set_v51_metadata():
    sample_fields = [
        {
            "name": "timestamp",
            "ftype": "fiftyone.core.fields.DateTimeField",
            "embedded_doc_type": None,
            "subfield": None,
            "fields": [],
            "db_field": "timestamp",
            "description": None,
            "info": None,
            "read_only": True,
            "created_at": {
                "$date": "2024-11-14T15:36:12.758Z"
            }
        },
        {
            "name": "sensor",
            "ftype": "fiftyone.core.fields.StringField",
            "embedded_doc_type": None,
            "subfield": None,
            "fields": [],
            "db_field": "sensor",
            "description": None,
            "info": None,
            "read_only": True,
            "created_at": {
                "$date": "2024-11-14T15:36:12.758Z"
            }
        }
    ]
    
    v51_metadata = {}
    v51_metadata["name"] = "Data Engine Rolling Dataset"  
    #v51_metadata["sample_fields"] = sample_fields

    file_path = os.path.join(storage_target_root, "metadata.json")
    with open(file_path, 'w') as json_file:
        json.dump(v51_metadata, json_file)

set_v51_metadata()

In [105]:
# Unpack data
# Support V51 dataset structure https://docs.voxel51.com/user_guide/dataset_creation/datasets.html#fiftyonedataset

def unpack_data():
    v51_samples = {}
    v51_samples_array = []
    if download_successful:
        for camera in tqdm(cameras_dict, desc="Unpacking data"):
            for aws_source in cameras_dict[camera]["aws-sources"]:
                for date in cameras_dict[camera]["aws-sources"][aws_source]:
                    for file in cameras_dict[camera]["aws-sources"][aws_source][date]:
                        file_path = os.path.join(data_target, file)
                        unpacking_successful = True
                        with open(file_path, 'r') as file:
                            for line in file:
                                try:
                                    data = json.loads(line)
                                    v51_sample = {}
                                    if "time" in data and "data" in data:
                                        # Get data
                                        timestamp = data.get("time")
                                        image_base64 = data.get("data")

                                        # Get timestamp
                                        time_obj = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
                                        formatted_time = time_obj.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                                        
                                    elif "image" in data and "sensor_name" in data and "event_timestamp" in data:
                                        # Get data
                                        image_base64 = data.get("image")
                                        sensor_name = data.get("sensor_name")
                                        timestamp = data.get("event_timestamp")
                                        
                                        # Get timestamps in UTC and Michigan time
                                        utc_time = datetime.datetime.fromtimestamp(timestamp, tz=datetime.timezone.utc)
                                        michigan_tz = pytz.timezone('America/Detroit')
                                        michigan_time = utc_time.astimezone(michigan_tz)
                                        formatted_time = michigan_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                                        # TODO Make sure that the conversion to Michigan time is correct
                                    else:
                                        unpacking_successful = False
                                        print(f"Format cannot be processed: {data}")
                                        continue
                                    
                                    if image_base64 and formatted_time:
                                        # Decode the base64 image data
                                        image_data = base64.b64decode(image_base64)

                                        # File paths
                                        image_filename = f"{camera}_{formatted_time}.jpg"
                                        output_path = os.path.join(data_target, image_filename)

                                        # Prepare import with V51
                                        v51_sample["filepath"] = output_path
                                        v51_sample["sensor"] = camera
                                        # Ensure milliseconds are in 3 digits
                                        milliseconds = formatted_time.split('.')[1][:3].ljust(3, '0')
                                        formatted_time = formatted_time.split('.')[0] + '.' + milliseconds + 'Z'
                                        iso8601_regex = re.compile(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(\.\d{3})?Z$')
                                        iso8601_conform = bool(iso8601_regex.match(formatted_time))
                                        if not iso8601_conform:
                                            print(f"Timestamp does not conform to ISO8601: {formatted_time}")
                                        v51_sample["timestamp"] = {"$date": formatted_time}
                                        v51_samples_array.append(v51_sample) 

                                        # Save the decoded image data as a JPEG
                                        with open(output_path, 'wb') as image_file:
                                            image_file.write(image_data)
                                    else:
                                        unpacking_successful = False
                                        print(f"There was an issue during file processing of {file}")
                                        continue

                                except json.JSONDecodeError as e:
                                    unpacking_successful = False
                                    print(f"Error decoding JSON: {e}")
                        
                        # Delete the original file if unpacking was successful
                        #if unpacking_successful:
                        #    os.remove(file_path)

    # Store V51 metadata files
    v51_samples["samples"] = v51_samples_array
    file_path = os.path.join(storage_target_root, "samples.json")
    with open(file_path, 'w') as json_file:
        json.dump(v51_samples, json_file)

unpack_data()

Unpacking data: 100%|██████████| 24/24 [00:05<00:00,  4.18it/s]


In [162]:
def create_v51_dataset(name="Data Engine Rolling Dataset", delete_old_dataset=False):

    # Create the dataset
    dataset = fo.Dataset(name = name, overwrite=delete_old_dataset)

    dataset.add_dir(
        dataset_dir=storage_target_root,
        dataset_type=fo.types.FiftyOneDataset,
        progress=True,
    )

    # View summary info about the dataset
    print(dataset)

    # Print the first few samples in the dataset
    print(dataset.head())
    return dataset

v51_dataset = create_v51_dataset(delete_old_dataset=True)

Importing samples...
 100% |█████████████| 27096/27096 [306.1ms elapsed, 0s remaining, 88.9K samples/s]   
Name:        Data Engine Rolling Dataset
Media type:  None
Num samples: 27096
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    sensor:           fiftyone.core.fields.StringField
    timestamp:        fiftyone.core.fields.DateTimeField
[<Sample: {
    'id': '673643dcdfd70ada14caee50',
    'media_type': None,
    'filepath': '/media/dbogdoll/Datasets/data_engine_rolling/data/plymouth_bishop_2023-03-13T14:56:11.419Z.jpg',
    'tags': [],
    'metadata': None,
    'created_at'

In [163]:
NUM_WORKERS=32
v51_dataset.compute_metadata(num_workers=NUM_WORKERS, progress=True)

Computing metadata...
 100% |█████████████| 27096/27096 [1.4s elapsed, 0s remaining, 24.7K samples/s]         


In [166]:
session = fo.launch_app(v51_dataset)